In [1]:
!pip install qrcode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.5 MB/s eta 0:00:00


In [2]:
!apt install libzbar0

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts imagemagick-6-common
  libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9 libgs9-common libidn12 libijs-0.35
  libjbig2dec0 libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6 libmagickcore-6.q16-6-extra
  libmagickwand-6.q16-6 libv4l-0 libv4lconvert0 libwmflite-0.2-7 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x libfftw3-bin
  libfftw3-dev inkscape poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts imagemagick-6-common
  libdjvulibre-text libdjvulibre21 libfftw3-doub

In [3]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.1 MB/s eta 0:00:00


In [4]:
!pip install pyzbar

In [5]:
import random
import json
import qrcode
from pyzbar.pyzbar import decode
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Hash import HMAC, SHA256
import base64
def manual_random_bytes(n):
    """Generates a list of n random bytes without using os.urandom()."""
    return bytes([random.randint(0, 255) for _ in range(n)])

def manual_base64_encode(data):
    """Manually encodes bytes to a base64-like string (limited approach)."""
    chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
    encoded = ""
    for i in range(0, len(data), 3):
        chunk = data[i:i+3]
        bin_str = ''.join(f'{byte:08b}' for byte in chunk)
        while len(bin_str) % 6 != 0:
            bin_str += '0'
        encoded += ''.join(chars[int(bin_str[i:i+6], 2)] for i in range(0, len(bin_str), 6))
    return encoded

def manual_base64_decode(data):
    """Manually decodes a base64-like string back to bytes (limited approach)."""
    chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
    binary_str = "".join(f'{chars.index(c):06b}' for c in data if c in chars)
    byte_data = bytes(int(binary_str[i:i+8], 2) for i in range(0, len(binary_str), 8))
    return byte_data

def generate_keys():
    """Generate AES and HMAC keys."""
    aes_key = manual_random_bytes(32)  # AES-256 key
    hmac_key = manual_random_bytes(32)  # HMAC-SHA256 key
    return aes_key, hmac_key

def encrypt_data(data, aes_key, hmac_key):
    """Encrypts data using AES-GCM and generates HMAC-SHA256 for integrity."""
    nonce = manual_random_bytes(12)  # 96-bit IV for AES-GCM
    cipher = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    ciphertext, auth_tag = cipher.encrypt_and_digest(data.encode())

    # Generate HMAC-SHA256
    hmac = HMAC.new(hmac_key, ciphertext + nonce, SHA256).digest()

    encrypted_output = {
        "ciphertext": base64.b64encode(ciphertext).decode('utf-8'), # Using base64.b64encode
        "auth_tag": base64.b64encode(auth_tag).decode('utf-8'), # Using base64.b64encode
        "nonce": base64.b64encode(nonce).decode('utf-8'), # Using base64.b64encode
        "hmac": base64.b64encode(hmac).decode('utf-8') # Using base64.b64encode
    }

    print("Ciphertext:", encrypted_output["ciphertext"])
    print("Nonce:", encrypted_output["nonce"])
    print("Authentication Tag:", encrypted_output["auth_tag"])
    print("HMAC Hash:", encrypted_output["hmac"])

    return encrypted_output

def generate_qr_code(encrypted_data, filename="secure_qr.png"):
    """Generates a QR code from encrypted data."""
    qr = qrcode.QRCode(version=1, box_size=10, border=5)
    qr.add_data(json.dumps(encrypted_data))
    qr.make(fit=True)
    img = qr.make_image(fill="black", back_color="white")
    img.save(filename)
    print(f"QR Code saved as {filename}")

def scan_qr_code(filename):
    """Scans and extracts data from a QR code."""
    img = Image.open(filename)
    decoded_data = decode(img)
    if decoded_data:
        return json.loads(decoded_data[0].data.decode())
    return None

def decrypt_data(encrypted_data, aes_key, hmac_key):
    """Decrypts AES-GCM encrypted data and verifies HMAC-SHA256."""
    ciphertext = base64.b64decode(encrypted_data["ciphertext"]) # Using base64.b64decode
    auth_tag = base64.b64decode(encrypted_data["auth_tag"]) # Using base64.b64decode
    nonce = base64.b64decode(encrypted_data["nonce"]) # Using base64.b64decode
    hmac_received = base64.b64decode(encrypted_data["hmac"]) # Using base64.b64decode
    # Verify HMAC
    hmac_calculated = HMAC.new(hmac_key, ciphertext + nonce, SHA256).digest()

    print("HMAC Received:", hmac_received.hex())
    print("HMAC Calculated:", hmac_calculated.hex())

    if hmac_received != hmac_calculated:
        raise ValueError("HMAC verification failed! QR Code may be tampered.")

    # Decrypt
    cipher = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    decrypted_data = cipher.decrypt_and_verify(ciphertext, auth_tag)
    return decrypted_data.decode()

# Example Usage
aes_key, hmac_key = generate_keys()
secret_message = input("Enter the secret message to encrypt: ")

# Encrypt and generate QR code
encrypted_data = encrypt_data(secret_message, aes_key, hmac_key)
generate_qr_code(encrypted_data)

# Scan and decrypt
encrypted_data_scanned = scan_qr_code("secure_qr.png")
if encrypted_data_scanned:
    decrypted_message = decrypt_data(encrypted_data_scanned, aes_key, hmac_key)
    print("Decrypted Data:", decrypted_message)


Enter the secret message to encrypt: hello
Ciphertext: 7uszzp4=
Nonce: vQziRNRcPp4TSQpZ
Authentication Tag: M9zzZgQdoA4PX4qanwmjXg==
HMAC Hash: iiR2AqJ5M+fTTniEvvwqUNy6U1AvIgYTzvlS3iGS2Vo=
QR Code saved as secure_qr.png
HMAC Received: 8a247602a27933e7d34e7884befc2a50dcba53502f220613cef952de2192d95a
HMAC Calculated: 8a247602a27933e7d34e7884befc2a50dcba53502f220613cef952de2192d95a
Decrypted Data: hello


In [ ]:
!pip install pyzbar

In [6]:
import random
import json
import qrcode
import base64
import math
from pyzbar.pyzbar import decode
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Hash import HMAC, SHA256
import io
import os

def manual_random_bytes(n):
    """Generates n random bytes manually."""
    return bytes([random.randint(0, 255) for _ in range(n)])

def compress_image(image_path, max_size_kb=100):
    """Compresses the image to reduce size before encryption."""
    img = Image.open(image_path)
    img.thumbnail((300, 300))  # Resize the image

    # Convert to RGB if the image has an alpha channel
    if img.mode == 'RGBA':
        img = img.convert('RGB')

    # Save to bytes with compression
    img_bytes = io.BytesIO()
    img.save(img_bytes, format="JPEG", quality=50)  # Reduce quality
    img_data = img_bytes.getvalue()

    return img_data

def generate_keys():
    """Generate AES and HMAC keys."""
    aes_key = manual_random_bytes(32)  # AES-256 key
    hmac_key = manual_random_bytes(32)  # HMAC-SHA256 key
    return aes_key, hmac_key

def encrypt_image(image_path, aes_key, hmac_key):
    """Encrypts an image and returns encrypted JSON data."""
    img_data = compress_image(image_path)

    nonce = manual_random_bytes(12)
    cipher = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    ciphertext, auth_tag = cipher.encrypt_and_digest(img_data)

    hmac = HMAC.new(hmac_key, ciphertext + nonce, SHA256).digest()

    encrypted_output = {
        "ciphertext": base64.b64encode(ciphertext).decode('utf-8'),
        "auth_tag": base64.b64encode(auth_tag).decode('utf-8'),
        "nonce": base64.b64encode(nonce).decode('utf-8'),
        "hmac": base64.b64encode(hmac).decode('utf-8')
    }

    return json.dumps(encrypted_output)

def split_data(data, chunk_size):
    """Splits large data into chunks of `chunk_size` bytes."""
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

def generate_multi_qr(encrypted_data, max_chunk_size=1000):
    """Splits encrypted data into chunks and generates multiple QR codes."""
    chunks = split_data(encrypted_data, max_chunk_size)
    total_parts = len(chunks)

    for i, chunk in enumerate(chunks):
        chunk_data = json.dumps({"part": i + 1, "total": total_parts, "data": chunk})
        qr = qrcode.QRCode(version=None, error_correction=qrcode.constants.ERROR_CORRECT_L, box_size=10, border=5)
        qr.add_data(chunk_data)
        qr.make(fit=True)
        img = qr.make_image(fill="black", back_color="white")
        filename = f"secure_qr_part_{i+1}.png"
        img.save(filename)
        print(f"QR Code {i+1}/{total_parts} saved as {filename}")

def scan_qr_codes(qr_files):
    """Scans multiple QR codes and reconstructs the full encrypted data."""
    scanned_parts = {}

    for qr_file in qr_files:
        img = Image.open(qr_file)
        decoded_data = decode(img)
        if decoded_data:
            part_info = json.loads(decoded_data[0].data.decode())
            scanned_parts[part_info["part"]] = part_info["data"]

    if len(scanned_parts) == 0:
        print("No valid QR codes found!")
        return None

    total_parts = max(scanned_parts.keys())

    # Check if all parts are present
    if len(scanned_parts) != total_parts:
        print(f"Missing QR parts! Expected {total_parts}, but got {len(scanned_parts)}.")
        return None

    # Reconstruct full encrypted data
    full_data = "".join(scanned_parts[i] for i in range(1, total_parts + 1))
    return json.loads(full_data)  # Convert back to dictionary

def decrypt_image(encrypted_data, aes_key, hmac_key, output_path="decrypted_image.jpg"):
    """Decrypts AES-GCM encrypted image data and verifies HMAC-SHA256."""
    ciphertext = base64.b64decode(encrypted_data["ciphertext"])
    auth_tag = base64.b64decode(encrypted_data["auth_tag"])
    nonce = base64.b64decode(encrypted_data["nonce"])
    hmac_received = base64.b64decode(encrypted_data["hmac"])

    hmac_calculated = HMAC.new(hmac_key, ciphertext + nonce, SHA256).digest()

    if hmac_received != hmac_calculated:
        raise ValueError("HMAC verification failed! QR Code may be tampered.")

    cipher = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    decrypted_data = cipher.decrypt_and_verify(ciphertext, auth_tag)

    with open(output_path, "wb") as img_file:
        img_file.write(decrypted_data)

    print(f"Decrypted image saved as {output_path}")

# Example Usage
aes_key, hmac_key = generate_keys()
image_path = input("Enter the image path to encrypt: ")

# Encrypt and generate multiple QR codes
encrypted_data = encrypt_image(image_path, aes_key, hmac_key)
generate_multi_qr(encrypted_data)

# Scan QR codes and reconstruct
qr_files = [f for f in os.listdir() if f.startswith("secure_qr_part_")]
encrypted_data_scanned = scan_qr_codes(qr_files)

if encrypted_data_scanned:
    decrypt_image(encrypted_data_scanned, aes_key, hmac_key)


Enter the image path to encrypt: /content/Ram Charan Latest Pics_16.jpg
QR Code 1/9 saved as secure_qr_part_1.png
QR Code 2/9 saved as secure_qr_part_2.png
QR Code 3/9 saved as secure_qr_part_3.png
QR Code 4/9 saved as secure_qr_part_4.png
QR Code 5/9 saved as secure_qr_part_5.png
QR Code 6/9 saved as secure_qr_part_6.png
QR Code 7/9 saved as secure_qr_part_7.png
QR Code 8/9 saved as secure_qr_part_8.png
QR Code 9/9 saved as secure_qr_part_9.png
Decrypted image saved as decrypted_image.jpg


In [ ]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.3 MB/s eta 0:00:00


In [ ]:
import random
import json
import qrcode
import base64
import io
import os
from pyzbar.pyzbar import decode
from PIL import Image
from Crypto.Cipher import AES
from Crypto.Hash import HMAC, SHA256

# --------------- Helper Functions ---------------
def manual_random_bytes(n):
    """Generates n random bytes manually."""
    return bytes([random.randint(0, 255) for _ in range(n)])

def split_data(data, chunk_size):
    """Splits large data into chunks of `chunk_size` bytes."""
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

# --------------- AES-GCM Encryption ---------------
def generate_keys():
    """Generate AES and HMAC keys."""
    aes_key = manual_random_bytes(32)  # AES-256 key
    hmac_key = manual_random_bytes(32)  # HMAC-SHA256 key
    return aes_key, hmac_key

def encrypt_video(video_path, aes_key, hmac_key):
    """Encrypts a video file without quality loss and returns encrypted JSON data."""
    with open(video_path, "rb") as f:
        video_data = f.read()

    nonce = manual_random_bytes(12)
    cipher = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    ciphertext, auth_tag = cipher.encrypt_and_digest(video_data)

    hmac = HMAC.new(hmac_key, ciphertext + nonce, SHA256).digest()

    encrypted_output = {
        "ciphertext": base64.b64encode(ciphertext).decode('utf-8'),
        "auth_tag": base64.b64encode(auth_tag).decode('utf-8'),
        "nonce": base64.b64encode(nonce).decode('utf-8'),
        "hmac": base64.b64encode(hmac).decode('utf-8')
    }

    return json.dumps(encrypted_output)

# --------------- QR Code Generation ---------------
def generate_multi_qr_video(encrypted_data, max_chunk_size=1000):
    """Splits encrypted video data into chunks and generates multiple QR codes."""
    chunks = split_data(encrypted_data, max_chunk_size)
    total_parts = len(chunks)

    for i, chunk in enumerate(chunks):
        chunk_data = json.dumps({"part": i + 1, "total": total_parts, "data": chunk})
        qr = qrcode.QRCode(version=None, error_correction=qrcode.constants.ERROR_CORRECT_L, box_size=10, border=5)
        qr.add_data(chunk_data)
        qr.make(fit=True)
        img = qr.make_image(fill="black", back_color="white")
        filename = f"secure_video_qr_part_{i+1}.png"
        img.save(filename)
        print(f"QR Code {i+1}/{total_parts} saved as {filename}")

# --------------- QR Code Scanning & Reconstruction ---------------
def scan_qr_codes(qr_files):
    """Scans multiple QR codes and reconstructs the full encrypted data."""
    scanned_parts = {}

    for qr_file in qr_files:
        img = Image.open(qr_file)
        decoded_data = decode(img)
        if decoded_data:
            part_info = json.loads(decoded_data[0].data.decode())
            scanned_parts[part_info["part"]] = part_info["data"]

    if len(scanned_parts) == 0:
        print("No valid QR codes found!")
        return None

    total_parts = max(scanned_parts.keys())

    # Check if all parts are present
    if len(scanned_parts) != total_parts:
        print(f"Missing QR parts! Expected {total_parts}, but got {len(scanned_parts)}.")
        return None

    # Reconstruct full encrypted data
    full_data = "".join(scanned_parts[i] for i in range(1, total_parts + 1))
    return json.loads(full_data)  # Convert back to dictionary

# --------------- AES-GCM Decryption ---------------
def decrypt_video(encrypted_data, aes_key, hmac_key, output_path="decrypted_video.mp4"):
    """Decrypts AES-GCM encrypted video data and verifies HMAC-SHA256."""
    ciphertext = base64.b64decode(encrypted_data["ciphertext"])
    auth_tag = base64.b64decode(encrypted_data["auth_tag"])
    nonce = base64.b64decode(encrypted_data["nonce"])
    hmac_received = base64.b64decode(encrypted_data["hmac"])

    hmac_calculated = HMAC.new(hmac_key, ciphertext + nonce, SHA256).digest()

    if hmac_received != hmac_calculated:
        raise ValueError("HMAC verification failed! QR Code may be tampered.")

    cipher = AES.new(aes_key, AES.MODE_GCM, nonce=nonce)
    decrypted_data = cipher.decrypt_and_verify(ciphertext, auth_tag)

    with open(output_path, "wb") as video_file:
        video_file.write(decrypted_data)

    print(f"Decrypted video saved as {output_path}")

# --------------- Example Usage ---------------
if __name__ == "__main__":
    aes_key, hmac_key = generate_keys()
    video_path = input("Enter the video path to encrypt: ")

    # Encrypt video without quality reduction
    encrypted_data = encrypt_video(video_path, aes_key, hmac_key)

    # Generate QR codes from encrypted video data
    generate_multi_qr_video(encrypted_data)

    # Scan QR codes and reconstruct encrypted data
    qr_files = [f for f in os.listdir() if f.startswith("secure_video_qr_part_")]
    encrypted_data_scanned = scan_qr_codes(qr_files)

    if encrypted_data_scanned:
        decrypt_video(encrypted_data_scanned, aes_key, hmac_key)


Streaming output truncated to the last 5000 lines.
QR Code 552/5550 saved as secure_video_qr_part_552.png
QR Code 553/5550 saved as secure_video_qr_part_553.png
QR Code 554/5550 saved as secure_video_qr_part_554.png
QR Code 555/5550 saved as secure_video_qr_part_555.png
QR Code 556/5550 saved as secure_video_qr_part_556.png
QR Code 557/5550 saved as secure_video_qr_part_557.png
QR Code 558/5550 saved as secure_video_qr_part_558.png
QR Code 559/5550 saved as secure_video_qr_part_559.png
QR Code 560/5550 saved as secure_video_qr_part_560.png
QR Code 561/5550 saved as secure_video_qr_part_561.png
QR Code 562/5550 saved as secure_video_qr_part_562.png
QR Code 563/5550 saved as secure_video_qr_part_563.png
QR Code 564/5550 saved as secure_video_qr_part_564.png
QR Code 565/5550 saved as secure_video_qr_part_565.png
QR Code 566/5550 saved as secure_video_qr_part_566.png
QR Code 567/5550 saved as secure_video_qr_part_567.png
QR Code 568/5550 saved as secure_video_qr_part_568.png
QR Code 569/55